# Amplitude Sketching Framework - Interactive Tutorial

This notebook demonstrates the unified amplitude sketching framework and shows how to use the base class.

In [ ]:
import numpy as np
import sys
sys.path.append('..')
from sim.amplitude_sketch import AmplitudeSketch, SerialComposition
from qiskit import QuantumCircuit

## 1. Creating a Simple Amplitude Sketch

Let's create a minimal concrete implementation.

In [ ]:
class SimpleSketch(AmplitudeSketch):
    """Minimal amplitude sketch for demonstration."""
    
    def __init__(self, m=16, k=3, theta=np.pi/4):
        super().__init__(m, k, theta)
        self.items = set()
    
    def insert(self, x: bytes):
        """Insert item x."""
        self.items.add(x)
        self.n_inserts += 1
    
    def query(self, y: bytes, shots=512, noise_level=0.0):
        """Query for item y."""
        circuit = self._build_insert_circuit(y)
        return self._measure_overlap(circuit, shots, noise_level)
    
    def _build_insert_circuit(self, x: bytes):
        """Build circuit with Rz rotations at hashed positions."""
        qc = QuantumCircuit(self.m)
        indices = self._hash_to_indices(x)
        for idx in indices:
            qc.rz(self.theta, idx)
        return qc

## 2. Basic Usage

In [ ]:
# Initialize sketch
sketch = SimpleSketch(m=16, k=3, theta=np.pi/4)
print(f"Initialized: {sketch}")

# Insert items
items = [b"apple", b"banana", b"cherry"]
for item in items:
    sketch.insert(item)

print(f"\nInserted {sketch.n_inserts} items")

## 3. Query Operations

In [ ]:
# Query for members
print("\nQuerying for members:")
for item in items:
    overlap = sketch.query(item, shots=512)
    print(f"  {item.decode()}: overlap = {overlap:.3f}")

# Query for non-members
print("\nQuerying for non-members:")
non_members = [b"dog", b"elephant"]
for item in non_members:
    overlap = sketch.query(item, shots=512)
    print(f"  {item.decode()}: overlap = {overlap:.3f}")

## 4. Error Bounds and Statistics

In [ ]:
# Get error bounds
alpha, beta = sketch.error_bound()
print(f"\nError bounds:")
print(f"  False positive rate (α): {alpha:.4f}")
print(f"  False negative rate (β): {beta:.4f}")

# Get statistics
stats = sketch.get_stats()
print(f"\nSketch statistics:")
for key, value in stats.items():
    print(f"  {key}: {value}")

## 5. Noise Robustness

In [ ]:
# Test with different noise levels
noise_levels = [0.0, 0.001, 0.005, 0.01]

print("\nNoise robustness test (querying 'apple'):")
for noise in noise_levels:
    overlap = sketch.query(b"apple", shots=1024, noise_level=noise)
    print(f"  noise={noise:.4f}: overlap = {overlap:.3f}")

## 6. Serial Composition

Chain multiple sketches for multi-stage filtering.

In [ ]:
# Create two-stage pipeline
sketch1 = SimpleSketch(m=16, k=3)
sketch2 = SimpleSketch(m=16, k=3)

# Insert different items into each stage
for item in [b"apple", b"banana"]:
    sketch1.insert(item)
    sketch2.insert(item)

sketch1.insert(b"cherry")  # Only in stage 1

# Create composition
pipeline = SerialComposition([sketch1, sketch2])

print("\nSerial composition test:")
print(f"  Pipeline stages: {pipeline.n_stages}")

# Query through pipeline
test_items = [b"apple", b"cherry", b"dog"]
for item in test_items:
    score = pipeline.query(item, shots=512)
    print(f"  {item.decode()}: score = {score:.3f}")

## 7. Composed Error Bounds

In [ ]:
# Error bounds for composition
alpha_total, beta_total = pipeline.error_bound()

print("\nComposed error bounds:")
print(f"  Total false positive rate: {alpha_total:.4f}")
print(f"  Total false negative rate: {beta_total:.4f}")

# Compare to individual stages
alpha1, beta1 = sketch1.error_bound()
print(f"\nStage 1 error: α={alpha1:.4f}, β={beta1:.4f}")
print(f"Composition amplifies error as expected")

## 8. Memory Analysis

In [ ]:
# Memory footprint
print("\nMemory analysis:")
print(f"  Sketch 1: {sketch1.get_memory_size()} qubits")
print(f"  Sketch 2: {sketch2.get_memory_size()} qubits")
print(f"  Pipeline total: {pipeline.get_total_memory()} qubits (max)")

## 9. Circuit Visualization

In [ ]:
# Visualize insert circuit
circuit = sketch._build_insert_circuit(b"test")
print("\nInsert circuit for 'test':")
print(circuit)
print(f"\nCircuit depth: {circuit.depth()}")
print(f"Circuit size: {circuit.size()}")

## 10. Performance Comparison

Compare different parameter configurations.

In [ ]:
import time

configs = [
    {'m': 16, 'k': 3, 'theta': np.pi/4},
    {'m': 32, 'k': 4, 'theta': np.pi/4},
    {'m': 64, 'k': 5, 'theta': np.pi/8},
]

print("\nPerformance comparison:")
for config in configs:
    sketch = SimpleSketch(**config)
    
    # Insert items
    start = time.time()
    for i in range(10):
        sketch.insert(f"item{i}".encode())
    insert_time = time.time() - start
    
    # Query items
    start = time.time()
    for i in range(5):
        sketch.query(f"item{i}".encode(), shots=256)
    query_time = time.time() - start
    
    print(f"\n  Config: m={config['m']}, k={config['k']}")
    print(f"    Insert time: {insert_time:.3f}s")
    print(f"    Query time: {query_time:.3f}s")
    print(f"    Memory: {sketch.get_memory_size()} qubits")

## Summary

The `AmplitudeSketch` base class provides:
- **Unified interface** for all quantum data structures
- **Automatic error bounds** based on universal lower bounds
- **Built-in noise support** with standardized models
- **Circuit caching** for performance
- **Composability** through `SerialComposition`
- **Statistics tracking** for analysis

All existing structures (QAM, QHT, Q-Count, etc.) can be refactored to inherit from this base class with minimal changes.